In [1]:
import pandas as pd
import numpy as np
import mysql.connector
df_user = pd.read_csv('Order_history.csv')
df_rest = pd.read_csv('Rest_Info.csv')

In [7]:
df_user.head()

,customer_id,order_id,rest_id
0,User9,1,6487f4c6fc13ae161d900937
1,User9,2,6487f4c6fc13ae161d900915
2,User8,3,6487f4c6fc13ae161d900939
3,User1,4,6487f4c6fc13ae161d900946
4,User1,5,6487f4c6fc13ae161d9008fc


In [8]:
df_rest.head()

,rest_id,rest_name,zone,rating,cuisine,budget
0,6487f4c6fc13ae161d9008e3,Durgan-Haag,North,1.4,mexican,5
1,6487f4c6fc13ae161d9008e4,Jacobson Group,South,2.6,indian,5
2,6487f4c6fc13ae161d9008e5,"Hintz, Harris and Beer",South,3.6,italian,1
3,6487f4c6fc13ae161d9008e6,Kling LLC,West,1.2,italian,3
4,6487f4c6fc13ae161d9008e7,Beer-Kuhic,North,4.7,italian,5


In [12]:
df = df_user.merge(df_rest, on  = "rest_id", how= "left").sort_values(by= "order_id")
df.head(10)

,customer_id,order_id,rest_id,rest_name,zone,rating,cuisine,budget
0,User9,1,6487f4c6fc13ae161d900937,Rohan and Sons,West,4.6,indian,5
1,User9,2,6487f4c6fc13ae161d900915,"Frami, McGlynn and VonRueden",North,1.8,indian,3
2,User8,3,6487f4c6fc13ae161d900939,Bednar LLC,West,3.4,mexican,3
3,User1,4,6487f4c6fc13ae161d900946,"O'Kon, Rosenbaum and Walter",West,1.7,indian,3
4,User1,5,6487f4c6fc13ae161d9008fc,Gorczany and Sons,North,3.9,italian,1
5,User7,6,6487f4c6fc13ae161d900945,Brakus and Sons,South,5.0,indian,5
6,User2,7,6487f4c6fc13ae161d9008f3,Witting and Sons,South,2.2,indian,1
7,User9,8,6487f4c6fc13ae161d90090e,Mosciski-Bradtke,North,4.7,mexican,4
8,User4,9,6487f4c6fc13ae161d9008ea,Nikolaus Inc,West,2.0,indian,5
9,User7,10,6487f4c6fc13ae161d900940,"Weimann, Gusikowski and Stracke",East,4.3,italian,3


In [15]:
# Replace the placeholders with your MySQL connection details
connection = mysql.connector.connect(
    host='localhost',
    port = '3306',
    user='root',
    password='sw23',
    database='flink'
)
cursor = connection.cursor()
query = "SELECT * FROM pending_orders_table;"
cursor.execute(query)
result = cursor.fetchall()


In [16]:
for row in result:
    print(row)

('6487f4c6fc13ae161d9008e3', 5)
('6487f4c6fc13ae161d9008e4', 5)
('6487f4c6fc13ae161d9008e5', 5)
('6487f4c6fc13ae161d9008e6', 5)
('6487f4c6fc13ae161d9008e7', 5)
('6487f4c6fc13ae161d9008e8', 5)
('6487f4c6fc13ae161d9008e9', 5)
('6487f4c6fc13ae161d9008ea', 5)
('6487f4c6fc13ae161d9008eb', 5)
('6487f4c6fc13ae161d9008ec', 5)
('6487f4c6fc13ae161d9008ed', 5)
('6487f4c6fc13ae161d9008ee', 4)
('6487f4c6fc13ae161d9008ef', 5)
('6487f4c6fc13ae161d9008f0', 5)
('6487f4c6fc13ae161d9008f1', 5)
('6487f4c6fc13ae161d9008f2', 5)
('6487f4c6fc13ae161d9008f3', 5)
('6487f4c6fc13ae161d9008f4', 5)
('6487f4c6fc13ae161d9008f5', 5)
('6487f4c6fc13ae161d9008f6', 5)
('6487f4c6fc13ae161d9008f7', 5)
('6487f4c6fc13ae161d9008f8', 5)
('6487f4c6fc13ae161d9008f9', 5)
('6487f4c6fc13ae161d9008fa', 5)
('6487f4c6fc13ae161d9008fb', 5)
('6487f4c6fc13ae161d9008fc', 5)
('6487f4c6fc13ae161d9008fd', 5)
('6487f4c6fc13ae161d9008fe', 5)
('6487f4c6fc13ae161d9008ff', 5)
('6487f4c6fc13ae161d900900', 5)
('6487f4c6fc13ae161d900901', 5)
('6487f4

Next steps -> 
Convert MySQL table to pandas df
Add column names to the df